In [1]:
import os
import logging
import pickle
import pandas as pd
import spacy
from spacy.vectors import Vectors
from spacy.strings import StringStore
from nltk.corpus import stopwords

import gensim
from gensim import corpora, models, similarities
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

%matplotlib inline

In [2]:
os.getcwd()

'/home/ryohayama/Python/b13logy/nlp'

In [3]:
dictionary = corpora.Dictionary.load('serialized_data/nlp09_lemmatized_corpus.dict')
corpus = corpora.MmCorpus('serialized_data/nlp09_bow_corpus.mm')

In [4]:
print(corpus)

MmCorpus(13434 documents, 27077 features, 1270334 non-zero entries)


In [5]:
# Set up a tf-idf transforming model using the corpus
tfidf_model = models.TfidfModel(corpus)

In [6]:
print(tfidf_model)

TfidfModel(num_docs=13434, num_nnz=1270334)


In [7]:
# Transform corpus into tf-idf space
corpus_tfidf = tfidf_model[corpus]

In [8]:
len(corpus_tfidf)

13434

In [9]:
num_topics=7

In [10]:
%%time
# Transform tf-idf corpus with Latent Semantic Analysis
lsi = models.LsiModel(
            corpus=corpus_tfidf, 
            id2word=dictionary, 
            num_topics=num_topics
)

CPU times: user 9.22 s, sys: 3.77 s, total: 13 s
Wall time: 4.44 s


In [11]:
lsi.print_topics(5)

[(0,
  '0.166*"ms" + 0.156*"mm" + 0.154*"min" + 0.143*"search" + 0.134*"peptide" + 0.120*"set" + 0.110*"ion" + 0.108*"sample" + 0.107*"modification" + 0.105*"database"'),
 (1,
  '-0.223*"search" + 0.199*"mm" + 0.166*"min" + -0.162*"database" + -0.159*"modification" + -0.135*"tolerance" + -0.128*"maxquant" + -0.126*"set" + -0.119*"mascot" + -0.119*"version"'),
 (2,
  '0.224*"cell" + 0.124*"-pron-" + 0.120*"study" + 0.118*"proteomic" + 0.108*"identify" + 0.099*"cancer" + 0.096*"change" + -0.096*"mascot" + 0.095*"expression" + 0.094*"protein"'),
 (3,
  '-0.250*"maxquant" + -0.245*"mm" + 0.200*"ms" + -0.144*"°" + 0.139*"mascot" + 0.133*"ion" + 0.124*"scan" + -0.118*"c" + -0.117*"andromeda" + 0.115*"orbitrap"'),
 (4,
  '-0.364*"maxquant" + 0.316*"mascot" + -0.158*"andromeda" + 0.144*"discoverer" + -0.137*"lfq" + 0.133*"mm" + 0.130*"science" + -0.127*"quantification" + 0.125*"matrix" + -0.115*"perseus"')]

In [12]:
corpus_lsi = lsi[corpus_tfidf]

In [13]:
len(corpus_lsi)

13434

In [14]:
for doc in corpus_lsi[:3]:
    print(doc)

[(0, 0.2863634515116126), (1, 0.24171032600217704), (2, -0.030296234728777783), (3, -0.09261507952954949), (4, 0.06626770528972953), (5, 0.008721577056012847), (6, 0.0014781850908041408)]
[(0, 0.0730448113608419), (1, 0.02054618707884547), (2, 0.08762624888768519), (3, 0.04950189318669456), (4, 0.016338666018110827), (5, 0.09400272918582958), (6, -0.031015548784552045)]
[(0, 0.28498212278810114), (1, 0.23202019086935002), (2, -0.04583892369276718), (3, 0.042124672463871006), (4, -0.01181207677150549), (5, 0.12442584635941627), (6, -0.038407258929755085)]


In [15]:
lsi.save('serialized_data/nlp10_tfidf.lsi')

In [16]:
%%time
# Transform tf-idf corpus with Latent Semantic Analysis
lda = models.LdaModel(
            corpus=corpus_tfidf, 
            id2word=dictionary, 
            num_topics=num_topics
)

CPU times: user 7.58 s, sys: 57.6 ms, total: 7.64 s
Wall time: 7.65 s


In [24]:
lda.print_topics(7)

[(0,
  '0.003*"cho" + 0.001*"ir" + 0.001*"podocyte" + 0.001*"gp120" + 0.001*"ppi" + 0.001*"synaptic" + 0.001*"giardia" + 0.001*"gpi" + 0.001*"akt1" + 0.001*"il-33"'),
 (1,
  '0.004*"cell" + 0.003*"stress" + 0.003*"host" + 0.003*"response" + 0.003*"plant" + 0.003*"reveal" + 0.002*"resistance" + 0.002*"complex" + 0.002*"change" + 0.002*"mutant"'),
 (2,
  '0.003*"novel" + 0.003*"assembly" + 0.002*"metabolism" + 0.002*"reveal" + 0.002*"inflammatory" + 0.002*"transport" + 0.002*"mitochondrial" + 0.002*"gene" + 0.002*"disease" + 0.002*"methylation"'),
 (3,
  '0.002*"cell" + 0.002*"glycosylation" + 0.002*"redox" + 0.002*"glycopeptide" + 0.002*"glycan" + 0.002*"understand" + 0.002*"proteomic" + 0.002*"temporal" + 0.002*"tissue" + 0.002*"ovarian"'),
 (4,
  '0.004*"search" + 0.003*"protein" + 0.003*"identify" + 0.003*"modification" + 0.003*"-pron-" + 0.003*"database" + 0.003*"cell" + 0.003*"ms" + 0.003*"proteome" + 0.003*"peptide"'),
 (5,
  '0.003*"cll" + 0.003*"platelet" + 0.002*"hsp90" + 0.002

In [18]:
corpus_lda = lda[corpus_tfidf]

In [19]:
len(corpus_lda)

13434

In [20]:
for doc in corpus_lda[:3]:
    print(doc)

[(0, 0.0122563355), (1, 0.012730888), (2, 0.012258478), (3, 0.012326433), (4, 0.5642195), (5, 0.37391886), (6, 0.012289526)]
[(0, 0.025604108), (1, 0.025692), (2, 0.4645122), (3, 0.40713558), (4, 0.025745993), (5, 0.025656058), (6, 0.025654022)]
[(0, 0.016169365), (1, 0.016245287), (2, 0.016243411), (3, 0.016236937), (4, 0.016260434), (5, 0.9026532), (6, 0.016191347)]


In [21]:
lda.save('serialized_data/nlp10_tfidf_%d.lda' % num_topics)

In [22]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [23]:
pyLDAvis.gensim.prepare(lda, corpus_tfidf[:1000], dictionary)

/home/ryohayama/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.182527 -0.053845       1        1  50.176376
3     -0.000497 -0.015837       2        1  22.963276
5     -0.047291 -0.030985       3        1  12.523563
1      0.043103  0.135756       4        1   5.395803
6     -0.042100 -0.042110       5        1   4.311375
2     -0.029084  0.019072       6        1   2.617464
0     -0.106658 -0.012051       7        1   2.012138, topic_info=      Category       Freq           Term      Total  loglift  logprob
term                                                                 
291    Default  19.000000           cell  19.000000  30.0000  30.0000
3414   Default   6.000000       response   6.000000  29.0000  29.0000
2186   Default   6.000000           gene   6.000000  28.0000  28.0000
897    Default   6.000000          novel   6.000000  27.0000  27.0000
12417  Default   5.000000           role   5.000000  26.0000  26.0000
794    Default   5.000000         reveal   5.000000  25.0000  25.0000
3360   Default   5.000000        disease   5.000000  24.0000  24.0000
2502   Default   4.000000         stress   4.000000  23.0000  23.0000
9749   Default   3.000000            cll   3.000000  22.0000  22.0000
1573   Default   8.000000         change   8.000000  21.0000  21.0000
1236   Default   4.000000  mitochondrial   4.000000  20.0000  20.0000
52     Default   7.000000        complex   7.000000  19.0000  19.0000
2004   Default   8.000000         mascot   8.000000  18.0000  18.0000
6628   Default   6.000000       regulate   6.000000  17.0000  17.0000
700    Default   4.000000        involve   4.000000  16.0000  16.0000
901    Default  13.000000          study  13.000000  15.0000  15.0000
2644   Default   8.000000     expression   8.000000  14.0000  14.0000
804    Default  14.000000         -pron-  14.000000  13.0000  13.0000
2149   Default   3.000000           host   3.000000  12.0000  12.0000
1136   Default   5.000000      associate   5.000000  11.0000  11.0000
1312   Default   3.000000     metabolism   3.000000  10.0000  10.0000
5168   Default   4.000000  glycosylation   4.000000   9.0000   9.0000
893    Default  14.000000       identify  14.000000   8.0000   8.0000
4935   Default   3.000000         kinase   3.000000   7.0000   7.0000
3609   Default   7.000000        pathway   7.000000   6.0000   6.0000
183    Default   8.000000         cancer   8.000000   5.0000   5.0000
1430   Default   3.000000       platelet   3.000000   4.0000   4.0000
721    Default  14.000000      proteomic  14.000000   3.0000   3.0000
9196   Default   5.000000     understand   5.000000   2.0000   2.0000
3373   Default   4.000000      infection   4.000000   1.0000   1.0000
...        ...        ...            ...        ...      ...      ...
11916   Topic7   0.166500         anchor   0.266351   3.4362  -6.9289
10972   Topic7   0.148277         social   0.237537   3.4347  -7.0448
2192    Topic7   0.126272           cdk1   0.202595   3.4332  -7.2054
16549   Topic7   0.120856           kcc2   0.194946   3.4279  -7.2493
9961    Topic7   0.117540        simplex   0.190205   3.4246  -7.2771
16725   Topic7   0.124978    magnetosome   0.204497   3.4136  -7.2157
14801   Topic7   0.114153       pyrenoid   0.189765   3.3977  -7.3063
4235    Topic7   0.104085            hdl   0.175254   3.3849  -7.3987
26863   Topic7   0.243564          gp120   0.416626   3.3692  -6.5485
8547    Topic7   0.094363          nerve   0.164557   3.3499  -7.4967
6170    Topic7   0.103507           cdna   0.180643   3.3491  -7.4042
8883    Topic7   0.123930       schizont   0.217574   3.3431  -7.2242
5729    Topic7   0.094940            aha   0.167175   3.3402  -7.4906
16605   Topic7   0.131336          ddx54   0.232154   3.3363  -7.1661
4034    Topic7   0.092426      flowering   0.164967   3.3266  -7.5175
13153   Topic7   0.083499            caf   0.152811   3.3016  -7.6190
10954   Topic7   0.081601           ev71 